In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-02-14 16:00:19.107407: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 16:00:19.249818: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/usr/lib/x86_64-linux-gnu/gazebo-9/plugins
2023-02-14 16:00:19.249842: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 16:00:20.067058: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [2]:
dataset = "./data/dataset.csv"
model_save_path = "./model/model.hdf5"

In [3]:
df = pd.read_csv(dataset, index_col=0)
NUM_CLASSES = df["label"].nunique()


# df.replace({"labels": {"up": 1, "down": 2, "left": 3, "right": 4, "forward": 5, "backward": 6}}, inplace=True)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=333)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((42, )),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='sigmoid'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='sigmoid'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

2023-02-14 16:00:21.555358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 16:00:21.555748: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/usr/lib/x86_64-linux-gnu/gazebo-9/plugins
2023-02-14 16:00:21.555829: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/usr/lib/x86_64-linux-gnu/gazebo-9/plugins
2023-02-14 16:00:21.555898: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load d

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 64)                2752      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0

In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), callbacks=[cp_callback, es_callback])

Epoch 1/1000
18/29 [=================>............] - ETA: 0s - loss: 2.3204 - accuracy: 0.1224 
Epoch 1: saving model to ./model/gesture_detector_final2.hdf5
29/29 [==============================] - 1s 13ms/step - loss: 2.2814 - accuracy: 0.1287 - val_loss: 2.1008 - val_accuracy: 0.3675
Epoch 2/1000
16/29 [===============>..............] - ETA: 0s - loss: 2.1345 - accuracy: 0.1587
Epoch 2: saving model to ./model/gesture_detector_final2.hdf5
29/29 [==============================] - 0s 6ms/step - loss: 2.1056 - accuracy: 0.1679 - val_loss: 1.9385 - val_accuracy: 0.4851
Epoch 3/1000
29/29 [==============================] - ETA: 0s - loss: 1.9774 - accuracy: 0.2521
Epoch 3: saving model to ./model/gesture_detector_final2.hdf5
29/29 [==============================] - 0s 7ms/step - loss: 1.9774 - accuracy: 0.2521 - val_loss: 1.7925 - val_accuracy: 0.6300
Epoch 4/1000
15/29 [==============>...............] - ETA: 0s - loss: 1.8735 - accuracy: 0.3318
Epoch 4: saving model to ./model/gesture_

In [10]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 2ms/step - loss: 0.0665 - accuracy: 0.9909


In [11]:
model = tf.keras.models.load_model(model_save_path)
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))


1/1 [==============================] - 0s 94ms/step
[9.6000093e-01 7.2310263e-06 3.2115639e-03 1.1640075e-02 3.9870506e-03
 8.6407039e-05 7.6887343e-05 2.0989848e-02]
0
